### 버스 정류장 최대한 축소하는 작업
- 승하차하는 승객 수, 그리고 일별 데이터 상관없이 정류장 하나에 모든 승객 수 합치고 중복데이터 삭제

In [ ]:
import pandas as pd
bus_graph = pd.read_csv('/content/drive/MyDrive/공모전 시각화/bus_graph.csv')
bus_graph.info()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
bus_graph.drop(['Unnamed: 0','정류소ID', '버스정류장ARS번호'], axis=1, inplace=True)

In [ ]:
bus_graph = bus_graph.dropna(subset=['노선번호'])

In [ ]:
bus_graph['노선번호'].nunique()

365

In [ ]:
bus_graph = bus_graph.dropna(subset=['구'])

In [ ]:
bus_graph.shape

(785712, 8)

In [ ]:
bus_graph.head()

,날짜,노선번호,승차총승객수,하차총승객수,정류소명,longitude3,latitude3,구
0,20230101,100,33,47,창경궁.서울대학교병원,126.996566,37.579183,종로구
1,20230101,107,4,16,창경궁.서울대학교병원,126.996566,37.579183,종로구
2,20230101,171,49,37,창경궁.서울대학교병원,126.996566,37.579183,종로구
3,20230101,172,38,34,창경궁.서울대학교병원,126.996566,37.579183,종로구
4,20230101,104,34,46,창경궁.서울대학교병원,126.996566,37.579183,종로구


---

#### **버스 정류소 데이터 정제 작업**

In [ ]:
a = bus_graph.groupby(['정류소명', 'longitude3', 'latitude3', '구'], as_index=False)[['승차총승객수', '하차총승객수']].agg('sum')

In [ ]:
a.head()

,정류소명,longitude3,latitude3,구,승차총승객수,하차총승객수
0,(구)경남호텔앞,127.070183,37.569605,동대문구,5070,20048
1,1156번종점,127.150751,37.650445,강남구,1223,13
2,1156번종점,127.151009,37.650561,강남구,9,1205
3,2001아울렛중계점,127.067663,37.641688,노원구,715,1680
4,508단지삼거리,127.005536,37.599333,성북구,36,9946


In [ ]:
a.shape

(6460, 6)

In [ ]:
# 정류소명을 기준으로 groupby
lat_long = bus_graph.groupby(['정류소명'], as_index=False)[['longitude3', 'latitude3', '승차총승객수', '하차총승객수']].agg('sum')# 6460개

In [ ]:
lat_long.head()

,정류소명,longitude3,latitude3,승차총승객수,하차총승객수
0,(구)경남호텔앞,15756.702692,4658.631020,5070,20048
1,1156번종점,7883.354551,2334.331176,1232,1218
2,2001아울렛중계점,3939.097559,1166.892335,715,1680
3,508단지삼거리,7874.346270,2331.155327,1194,9988
4,508단지입구,3937.197972,1165.574408,1483,110


In [ ]:
lat_long.shape

(4242, 5)

In [ ]:
# inner join 하고 나서, 위/경도 데이터는 lat_long으로 보존한다.
data = pd.merge(a, lat_long, on='정류소명', how='inner')

In [ ]:
data.shape

(6460, 10)

In [ ]:
data.head()

,정류소명,longitude3_x,latitude3_x,구,승차총승객수_x,하차총승객수_x,longitude3_y,latitude3_y,승차총승객수_y,하차총승객수_y
0,(구)경남호텔앞,127.070183,37.569605,동대문구,5070,20048,15756.702692,4658.631020,5070,20048
1,1156번종점,127.150751,37.650445,강남구,1223,13,7883.354551,2334.331176,1232,1218
2,1156번종점,127.151009,37.650561,강남구,9,1205,7883.354551,2334.331176,1232,1218
3,2001아울렛중계점,127.067663,37.641688,노원구,715,1680,3939.097559,1166.892335,715,1680
4,508단지삼거리,127.005536,37.599333,성북구,36,9946,7874.346270,2331.155327,1194,9988


In [ ]:
data  = data.drop_duplicates(subset=['정류소명']) # 정류소로 중복되는 데이터 제거

In [ ]:
# 승차총승객수_x	하차총승객수_x 는 제거
# longitude3_y	latitude3_y 는 제거
drop_cols = [
    '승차총승객수_x', '하차총승객수_x', 'longitude3_y', 'latitude3_y'
]

data.drop(drop_cols, axis=1, inplace=True)

In [ ]:
data.rename(columns = {
    '승차총승객수_y': '승차 총 승객수',
    '하차총승객수_y': '하차 총 승객수',
    'longitude3_x' : 'longitude3',
    'latitude3_x' : 'latitude3'
}, inplace=True)

In [ ]:
data.head()

,정류소명,longitude3,latitude3,구,승차 총 승객수,하차 총 승객수
0,(구)경남호텔앞,127.070183,37.569605,동대문구,5070,20048
1,1156번종점,127.150751,37.650445,강남구,1232,1218
3,2001아울렛중계점,127.067663,37.641688,노원구,715,1680
4,508단지삼거리,127.005536,37.599333,성북구,1194,9988
6,508단지입구,127.006386,37.599174,성북구,1483,110


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4242 entries, 0 to 6458
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   정류소명        4242 non-null   object 
 1   longitude3  4242 non-null   float64
 2   latitude3   4242 non-null   float64
 3   구           4242 non-null   object 
 4   승차 총 승객수    4242 non-null   int64  
 5   하차 총 승객수    4242 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 232.0+ KB


In [ ]:
cd /content/drive/MyDrive/공모전 시각화/

/content/drive/MyDrive/공모전 시각화


In [ ]:
data.to_csv('./버스정류소 승하차승객수.csv', index=False)

####  지하철 데이터 정제 작업(merge하기 위함)

In [ ]:
subway_graph = pd.read_csv('/content/drive/MyDrive/공모전 시각화/subway_graph.csv')
subway_graph.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4498 entries, 0 to 4497
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4498 non-null   int64  
 1   사용일자        4498 non-null   object 
 2   노선명         4498 non-null   object 
 3   역명          4498 non-null   object 
 4   승차총승객수      4498 non-null   int64  
 5   하차총승객수      4498 non-null   int64  
 6   요일          4498 non-null   object 
 7   latitude1   4498 non-null   float64
 8   longitude1  4498 non-null   float64
 9   구           4498 non-null   object 
 10  동           4498 non-null   object 
 11  아파트         4498 non-null   object 
 12  세부주소        4343 non-null   object 
 13  longitude2  4498 non-null   float64
 14  latitude2   4498 non-null   float64
 15  주소          4498 non-null   object 
 16  거리(km)      4498 non-null   float64
 17  정규화승차승객수    4498 non-null   float64
 18  정규화하차층객수    4498 non-null   float64
dtypes: float64(7), int64(3), ob

In [ ]:
drop_cols = ['Unnamed: 0', '사용일자', '노선명', '동', '세부주소', '주소', '정규화승차승객수', '정규화하차층객수', '요일']
subway_graph.drop(drop_cols, axis=1, inplace=True)

- 승차총승객수와 하차총승객수 데이터는 거리가 집까지의 거리가 1km내에 위치한 승하차총승객수를 산출한 값이다.
- lat1, lng1 은 각각 해당 역의 위경도 좌표값이다.
- lat2, lng2는 아파트에 대한 위경도 좌표값이다.
- 지하철역에서 해당 아파트까지의 거리가 거리(KM)칼럼에 들어있다. 

In [ ]:
subway_graph = subway_graph.drop_duplicates(subset=['역명', '아파트'])

In [ ]:
subway_graph.head()


,역명,승차총승객수,하차총승객수,latitude1,longitude1,구,아파트,longitude2,latitude2,거리(km)
0,신도림,20210,21037,37.508725,126.891295,구로구,신도림우성5차,126.883792,37.512723,0.875370
31,도림천,334,398,37.514287,126.882768,구로구,신도림우성5차,126.883792,37.512723,0.154363
62,합정,16903,16841,37.549463,126.913739,마포구,마포한강푸르지오,126.911963,37.550009,0.200648
93,이대,6428,7477,37.556733,126.946013,마포구,동양엔파트,126.940606,37.547947,0.839863
124,아현,4345,4510,37.557345,126.956141,마포구,서서울삼성아파트,126.959133,37.556362,0.338947


In [ ]:
subway_graph.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 4467
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   역명          146 non-null    object 
 1   승차총승객수      146 non-null    int64  
 2   하차총승객수      146 non-null    int64  
 3   latitude1   146 non-null    float64
 4   longitude1  146 non-null    float64
 5   구           146 non-null    object 
 6   아파트         146 non-null    object 
 7   longitude2  146 non-null    float64
 8   latitude2   146 non-null    float64
 9   거리(km)      146 non-null    float64
dtypes: float64(5), int64(2), object(3)
memory usage: 12.5+ KB


In [ ]:
final = pd.merge(data, subway_graph, on='구', how='inner')

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28847 entries, 0 to 28846
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   정류소명        28847 non-null  object 
 1   longitude3  28847 non-null  float64
 2   latitude3   28847 non-null  float64
 3   구           28847 non-null  object 
 4   승차 총 승객수    28847 non-null  int64  
 5   하차 총 승객수    28847 non-null  int64  
 6   역명          28847 non-null  object 
 7   승차총승객수      28847 non-null  int64  
 8   하차총승객수      28847 non-null  int64  
 9   latitude1   28847 non-null  float64
 10  longitude1  28847 non-null  float64
 11  아파트         28847 non-null  object 
 12  longitude2  28847 non-null  float64
 13  latitude2   28847 non-null  float64
 14  거리(km)      28847 non-null  float64
dtypes: float64(7), int64(4), object(4)
memory usage: 3.5+ MB


In [ ]:
final = final.dropna()

In [ ]:
final.head()

,정류소명,longitude3,latitude3,구,승차 총 승객수,하차 총 승객수,역명,승차총승객수,하차총승객수,latitude1,longitude1,아파트,longitude2,latitude2,거리(km)
0,1156번종점,127.150751,37.650445,강남구,1232,1218,선릉,9630,8387,37.504503,127.049008,대치대우아이빌4차,127.053887,37.505136,0.543877
1,1156번종점,127.150751,37.650445,강남구,1232,1218,대치,5513,4911,37.494612,127.063642,대치대우아이빌4차,127.053887,37.505136,1.292940
2,1156번종점,127.150751,37.650445,강남구,1232,1218,학여울,614,590,37.496663,127.070594,대치대우아이빌4차,127.053887,37.505136,1.941308
3,1156번종점,127.150751,37.650445,강남구,1232,1218,역삼,8548,9140,37.500622,127.036456,역삼휴먼터치빌,127.033829,37.506411,0.485123
4,1156번종점,127.150751,37.650445,강남구,1232,1218,신논현,12641,11595,37.504598,127.025060,역삼휴먼터치빌,127.033829,37.506411,0.982028


In [ ]:
final.rename(columns = {
    '승차 총 승객수' : '버스 승차 총 승객수',
    '하차 총 승객수' : '버스 하차 총 승객수',
    '승차총승객수' : '지하철 승차 총 승객수',
    '하차총승객수' : '지하철 하차 총 승객수'
}, inplace=True)

In [ ]:
final.head(10)

In [ ]:
final = final[final['거리(km)'] < 1]

In [ ]:
final.head(10)

,정류소명,longitude3,latitude3,구,버스 승차 총 승객수,버스 하차 총 승객수,역명,지하철 승차 총 승객수,지하철 하차 총 승객수,latitude1,longitude1,아파트,longitude2,latitude2,거리(km)
0,1156번종점,127.150751,37.650445,강남구,1232,1218,선릉,9630,8387,37.504503,127.049008,대치대우아이빌4차,127.053887,37.505136,0.543877
3,1156번종점,127.150751,37.650445,강남구,1232,1218,역삼,8548,9140,37.500622,127.036456,역삼휴먼터치빌,127.033829,37.506411,0.485123
4,1156번종점,127.150751,37.650445,강남구,1232,1218,신논현,12641,11595,37.504598,127.025060,역삼휴먼터치빌,127.033829,37.506411,0.982028
7,1156번종점,127.150751,37.650445,강남구,1232,1218,강남구청,3256,3385,37.517186,127.041280,논현동부센트레빌,127.037673,37.510359,0.607807
8,1156번종점,127.150751,37.650445,강남구,1232,1218,학동,3718,3964,37.514229,127.031656,논현동부센트레빌,127.037673,37.510359,0.717099
12,1156번종점,127.150751,37.650445,강남구,1232,1218,대청,3585,3794,37.493514,127.079532,일원청솔빌리지,127.083078,37.480481,0.958077
13,1156번종점,127.150751,37.650445,강남구,1232,1218,일원,3125,3229,37.483681,127.084390,일원청솔빌리지,127.083078,37.480481,0.259278
14,1156번종점,127.150751,37.650445,강남구,1232,1218,수서,9989,9358,37.487371,127.101880,수서동익,127.096467,37.485649,0.612486
15,1156번종점,127.150751,37.650445,강남구,1232,1218,청담,4808,5071,37.519365,127.053350,삼성청구타운,127.058378,37.519783,0.559470
16,KT강남지사,127.041465,37.504758,강남구,33444,33735,선릉,9630,8387,37.504503,127.049008,대치대우아이빌4차,127.053887,37.505136,0.543877


In [ ]:
cd /content/drive/MyDrive/공모전 시각화

/content/drive/MyDrive/공모전 시각화


In [ ]:
final.to_csv('./show_plot_final.csv', index=False)

#### 아래 데이터는 시도하지 말것. (그냥 이렇게 하면 그래프 못 그리는 구나 하고 생각하면 됩니다.)

- 2023년 2월 일별 버스정류장 데이터: 약 70만 건
- 자치구 별로 버스 정류장 데이터를 나누었음: 가장 적은 자치구의 경우 약 3만건의 데이터가 산출됨.
- 그럼에도, 그래프 그릴려고 하면 주피터 노트북 및 코랩 다운, 특히 메모리 부족으로 웹사이트 조차 로드되지 않는 현상 발생!
- 본인의 노트북 램이 32gb임에도 이 현상이 발생!

- 즉, 3만건 이상의 데이터는 시각화가 불가하니, 몇 천건 데이터로 정제하는 작업이 반드시 필요!

In [ ]:
# 데이터가 너무 커서 '구'별로 데이터 분리 후 따로 csv파일로 저장하겠다.

In [ ]:
bus_graph = bus_graph.dropna(subset=['구'])

In [ ]:
bus_graph['구'].unique()

array(['종로구', '용산구', '중구', '강남구', '서초구', '마포구', '은평구', '영등포구', '중랑구',
       '광진구', '송파구', '동대문구', '성동구', '동작구', '서대문구', '관악구', '강서구', '양천구',
       '구로구', '금천구', '노원구', '성북구', '강북구', '강동구', '도봉구'], dtype=object)

In [ ]:
# 1. 종로구
g_1 = bus_graph[bus_graph['구'] == '종로구']

# 2. 용산구
g_2 = bus_graph[bus_graph['구'] == '용산구']

# 3. 중구
g_3 = bus_graph[bus_graph['구'] == '중구']

# 4. 강남구
g_4 = bus_graph[bus_graph['구'] == '강남구']

# 5. 서초구
g_5 = bus_graph[bus_graph['구'] == '서초구']

# 6. 마포구
g_6 = bus_graph[bus_graph['구'] == '마포구']

# 7. 은평구
g_7 = bus_graph[bus_graph['구'] == '은평구']

# 8. 영등포구
g_8 = bus_graph[bus_graph['구'] == '영등포구']

# 9. 중랑구
g_9 = bus_graph[bus_graph['구'] == '중랑구']

# 10. 광진구
g_10 = bus_graph[bus_graph['구'] == '광진구']

In [ ]:
# 11. 송파구
g_11 = bus_graph[bus_graph['구'] == '송파구']
# 12. 동대문구
g_12 = bus_graph[bus_graph['구'] == '동대문구']
# 13. 성동구
g_13 = bus_graph[bus_graph['구'] == '성동구']
# 14. 동작구
g_14 = bus_graph[bus_graph['구'] == '동작구']
# 15. 서대문구
g_15 = bus_graph[bus_graph['구'] == '서대문구']
# 16. 관악구
g_16 = bus_graph[bus_graph['구'] == '관악구']
# 17. 강서구
g_17 = bus_graph[bus_graph['구'] == '강서구']
# 18. 양천구
g_18 = bus_graph[bus_graph['구'] == '양천구']
# 19. 구로구
g_19 = bus_graph[bus_graph['구'] == '구로구']
# 20. 금천구
g_20 = bus_graph[bus_graph['구'] == '금천구']
# 21. 노원구
g_21 = bus_graph[bus_graph['구'] == '노원구']
# 22. 성북구
g_22 = bus_graph[bus_graph['구'] == '성북구']
# 23. 강북구
g_23 = bus_graph[bus_graph['구'] == '강북구']
# 24. 강동구
g_24 = bus_graph[bus_graph['구'] == '강동구']
# 25. 도봉구
g_25 = bus_graph[bus_graph['구'] == '도봉구']

In [ ]:
g_1.shape

(28943, 6)

In [ ]:
g_1.head(2)

,노선번호,승차총승객수,하차총승객수,longitude3,latitude3,구
0,100,33,47,126.996566,37.579183,종로구
1,107,4,16,126.996566,37.579183,종로구


In [ ]:
### 집 주소도 마찬가지로 구 단위로 분류를 해준다.

In [ ]:
subway_graph = pd.read_csv('/content/drive/MyDrive/공모전 시각화/subway_graph.csv')
subway_graph.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4498 entries, 0 to 4497
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4498 non-null   int64  
 1   사용일자        4498 non-null   object 
 2   노선명         4498 non-null   object 
 3   역명          4498 non-null   object 
 4   승차총승객수      4498 non-null   int64  
 5   하차총승객수      4498 non-null   int64  
 6   요일          4498 non-null   object 
 7   latitude1   4498 non-null   float64
 8   longitude1  4498 non-null   float64
 9   구           4498 non-null   object 
 10  동           4498 non-null   object 
 11  아파트         4498 non-null   object 
 12  세부주소        4343 non-null   object 
 13  longitude2  4498 non-null   float64
 14  latitude2   4498 non-null   float64
 15  주소          4498 non-null   object 
 16  거리(km)      4498 non-null   float64
 17  정규화승차승객수    4498 non-null   float64
 18  정규화하차층객수    4498 non-null   float64
dtypes: float64(7), int64(3), ob

In [ ]:
drop_cols = ['Unnamed: 0', '사용일자', '노선명', '역명', 'latitude1', 'longitude1', '동', '세부주소', '주소', '정규화승차승객수', '정규화하차층객수']
subway_graph.drop(drop_cols, axis=1, inplace=True)

In [ ]:
subway_graph.head(2)

,승차총승객수,하차총승객수,요일,구,아파트,longitude2,latitude2,거리(km)
0,20210,21037,일,구로구,신도림우성5차,126.883792,37.512723,0.87537
1,45898,44849,월,구로구,신도림우성5차,126.883792,37.512723,0.87537


종로구', '용산구', '중구', '강남구', '서초구', '마포구', '은평구', '영등포구', '중랑구',
       '광진구', '송파구', '동대문구', '성동구', '동작구', '서대문구', '관악구', '강서구', '양천구',
       '구로구', '금천구', '노원구', '성북구', '강북구', '강동구', '도봉구'], dtype=object)

In [ ]:
a = subway_graph.copy()

# 1. 종로구
h_1 = a[a['구'] == '종로구']

# 2.용산구
h_2 = a[a['구'] == '용산구']

# 3.중구
h_3 = a[a['구'] == '중구']
#4.강남구
h_4 = a[a['구'] == '강남구']
#5. 서초구
h_5 = a[a['구'] == '서초구']
#6. 마포구
h_6 = a[a['구'] == '마포구']
#7. 은평구
h_7 = a[a['구'] == '은평구']
#8. 영등포구
h_8 = a[a['구'] == '영등포구']
#9. 중랑구
h_9 = a[a['구'] == '중랑구']
#10. 광진구
h_10 = a[a['구'] == '광진구']
# 11.송파구
h_11 = a[a['구'] == '송파구']
# 12.동대문구
h_12 = a[a['구'] == '동대문구']

# 13.성동구
h_13 = a[a['구'] == '성동구']
# 14.동작구
h_14 = a[a['구'] == '동작구']
# 15. 서대문구
h_15 = a[a['구'] == '서대문구']
# 16. 관악구
h_16 = a[a['구'] == '관악구']
# 17. 강서구
h_17 = a[a['구'] == '강서구']
# 18. 양천구
h_18 = a[a['구'] == '양천구']
# 19. 구로구
h_19 = a[a['구'] == '구로구']
# 20. 금천구
h_20 = a[a['구'] == '금천구']
# 21. 노원구
h_21 = a[a['구'] == '노원구']
# 22. 성북구
h_22 = a[a['구'] == '성북구']
# 23. 강북구
h_23 = a[a['구'] == '강북구']
# 24. 강동구
h_24 = a[a['구'] == '강동구']
# 25. 도봉구
h_25 = a[a['구'] == '도봉구']

In [ ]:
h_1.head()

,승차총승객수,하차총승객수,요일,구,아파트,longitude2,latitude2,거리(km)
496,4970,4640,일,종로구,롯데캐슬천지인,127.016189,37.573855,0.086398
497,6740,7148,월,종로구,롯데캐슬천지인,127.016189,37.573855,0.086398
498,7228,7441,화,종로구,롯데캐슬천지인,127.016189,37.573855,0.086398
499,7842,7980,수,종로구,롯데캐슬천지인,127.016189,37.573855,0.086398
500,7538,7850,목,종로구,롯데캐슬천지인,127.016189,37.573855,0.086398


In [ ]:
g_1.head(2)

,노선번호,승차총승객수,하차총승객수,longitude3,latitude3,구
0,100,33,47,126.996566,37.579183,종로구
1,107,4,16,126.996566,37.579183,종로구


### merge후 25개의 데이터 생성

종로구', '용산구', '중구', '강남구', '서초구', '마포구', '은평구', '영등포구', '중랑구', '광진구', '송파구', '동대문구', '성동구', '동작구', '서대문구', '관악구', '강서구', '양천구', '구로구', '금천구', '노원구', '성북구', '강북구', '강동구', '도봉구'], dtype=object)

In [ ]:
# 1.종로구
m_1 = pd.merge(g_1, h_1, on='구', how='left')

In [ ]:
# 2. 용산구
m_2 = pd.merge(g_2, h_2, on='구', how='left')
# 3. 중구


# 4. 강남구
m_4 = pd.merge(g_4, h_4, on='구', how='left')

# 5. 서초구
m_5 = pd.merge(g_5, h_5, on='구', how='left')
# 6. 마포구
m_6 = pd.merge(g_6, h_6, on='구', how='left')
# 7. 은평구
m_7 = pd.merge(g_7, h_7, on='구', how='left')
# 8. 영등포구

# 9. 중랑구
m_9 = pd.merge(g_9, h_9, on='구', how='left')
# 10. 광진구
m_10 = pd.merge(g_10, h_10, on='구', how='left')
# 11. 송파구
m_11 = pd.merge(g_11, h_11, on='구', how='left')
# 12. 동대문구

# 13. 성동구
m_13 = pd.merge(g_13, h_13, on='구', how='left')
# 14. 동작구
m_14 = pd.merge(g_14, h_14, on='구', how='left')
# 15. 서대문구

# 16. 관악구
m_16 = pd.merge(g_16, h_16, on='구', how='left')
# 17. 강서구
m_17 = pd.merge(g_17, h_17, on='구', how='left')
# 18. 양천구
m_18 = pd.merge(g_18, h_18, on='구', how='left')
# 19. 구로구
m_19 = pd.merge(g_19, h_19, on='구', how='left')
# 20. 금천구
m_20 = pd.merge(g_20, h_20, on='구', how='left')

# 21. 노원구
m_21 = pd.merge(g_21, h_21, on='구', how='left')

# 22. 성북구
m_22 = pd.merge(g_22, h_22, on='구', how='left')

# 23. 강북구
m_23 = pd.merge(g_23, h_23, on='구', how='left')

# 24. 강동구
m_24 = pd.merge(g_24, h_24, on='구', how='left')
# 25. 도봉구
m_25 = pd.merge(g_25, h_25, on='구', how='left')

### 지하철을 나와서 얼마만큼 버스를 타는지를 조사하고 싶다.

- 요일에 따라 승차승객수와 하차승객수를 알고 싶다.

In [ ]:
m_1.head()

In [ ]:
m_1.groupby('요일')[['승차총승객수_y', '하차총승객수_y']].agg('mean', 'sum')

,승차총승객수_y,하차총승객수_y
요일,,
금,6091.500000,5903.000000
목,6270.500000,6197.750000
수,6051.750000,5991.583333
월,5586.133333,5590.600000
일,4302.733333,4369.466667
토,5412.333333,5478.666667
화,5550.866667,5511.400000


In [ ]:
cd /content/drive/MyDrive/공모전 시각화

/content/drive/MyDrive/공모전 시각화


종로구', '용산구', '중구', '강남구', '서초구', '마포구', '은평구', '영등포구', '중랑구', '광진구', '송파구', '동대문구', '성동구', '동작구', '서대문구', '관악구', '강서구', '양천구', '구로구', '금천구', '노원구', '성북구', '강북구', '강동구', '도봉구

In [ ]:
m_1.to_csv('./종로구m.csv', index=False)

In [ ]:
m_2.to_csv('./용산구m.csv', index=False)
m_4.to_csv('./강남구m.csv', index=False)
m_5.to_csv('./서초구m.csv', index=False)
m_6.to_csv('./마포구m.csv', index=False)
m_7.to_csv('./은평구m.csv', index=False)
m_9.to_csv('./중랑구m.csv', index=False)
m_10.to_csv('./광진구m.csv', index=False)
m_11.to_csv('./송파구m.csv', index=False)
m_13.to_csv('./성동구m.csv', index=False)
m_14.to_csv('./동작구m.csv', index=False)
m_16.to_csv('./관악구m.csv', index=False)
m_17.to_csv('./강서구m.csv', index=False)
m_18.to_csv('./양천구m.csv', index=False)
m_19.to_csv('./구로구m.csv', index=False)
m_20.to_csv('./금천구m.csv', index=False)
m_21.to_csv('./노원구m.csv', index=False)
m_22.to_csv('./성북구m.csv', index=False)
m_23.to_csv('./강북구m.csv', index=False)
m_24.to_csv('./강동구m.csv', index=False)
m_25.to_csv('./도봉구m.csv', index=False)